<a href="https://colab.research.google.com/github/SrBliss/CometML-NLP_IMDB/blob/main/Comet_Movie_Reviews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Installing required packages

In [1]:
!pip install comet_ml

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.4/412.4 kB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.2/75.2 kB 15.2 MB/s eta 0:00:00
  Using cached requests-2.28.1-py3-none-any.whl (62 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.0/163.0 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.3/54.3 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 495.9/495.9 kB 54.6 MB/s eta 0:00:00
  Using cached urllib3-1.26.12-py2.py3-none-any.whl (140 kB)
  Preparing metadata (setup.py) ... done
  Using cached charset_normalizer-2.1.1-py3-none-any.whl (39 kB)
  Created wheel for configobj: filename=configobj-5.0.6-py3-none-any.whl size=34528 sha256=d86c2e1e51147a222d373f34980394eadab16a66ea3632d6e6e8b6db852e7c01
  Stored in directory: /home/bliss/.cache/pip/wheels/0d/c4/19/13d74440f2a571841db6b6e0a273694327498884dafb9cf978
Successfully built confi

## Importing required packages

In [4]:
# import logging
import os

from comet_ml import Experiment
from comet_ml import config
from comet_ml import API

import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds

In [ ]:
# os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

## Load data and create partitions (train, validation, test)

*This cell takes about 1 minute to execute*

In [5]:
# Load data
train_data, test_data = tfds.load(name="imdb_reviews",
                                  split=["train", "test"], 
                                  batch_size=-1, 
                                  as_supervised=True)

train_examples, train_labels = tfds.as_numpy(train_data)
test_examples, test_labels = tfds.as_numpy(test_data)
    
x_val = train_examples[:10000]
partial_x_train = train_examples[10000:]
y_val = train_labels[:10000]
partial_y_train = train_labels[10000:]

2022-10-15 14:49:23.757691: W tensorflow/core/platform/cloud/google_auth_provider.cc:184] All attempts to get a Google authentication bearer token failed, returning an empty token. Retrieving token from files failed with "NOT_FOUND: Could not locate the credentials file.". Retrieving token from GCE failed with "FAILED_PRECONDITION: Error executing an HTTP request: libcurl code 6 meaning 'Couldn't resolve host name', error details: Could not resolve host: metadata".


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

...

Shuffling /home/bliss/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteAHRZXU/imdb_reviews-test.tfr…

Generating unsupervised examples...:   0%|          | 0/50000 [00:00<?, ? examples/s]

Shuffling /home/bliss/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteAHRZXU/imdb_reviews-unsuperv…

Dataset imdb_reviews downloaded and prepared to /home/bliss/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


2022-10-15 14:50:06.453429: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-10-15 14:50:06.453539: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-10-15 14:50:06.453591: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2022-10-15 14:50:06.453633: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2022-10-15 14:50:06.453676: W tensorflow/stream_executor/platform/default/dso_loader.cc:6

## My Experiment: Build, Train and Track the Movie Reviews Model 

In [ ]:
# !cat /proc/cpuinfo

In [ ]:
# TO DO: API on config file or on environment variable
# os.environ['COMET_API_KEY'] = "EpnUTFvwqbgvmSpjWOTMBPlP5"

In [6]:
exp = Experiment (
    api_key="EpnUTFvwqbgvmSpjWOTMBPlP5",
    project_name="rmiranda-movie-reviews",
    workspace="rodrigomiranda21",
    # Some extra optional parameters below
    # auto_histogram_weight_logging=True,
    # log_code=True,
    )

COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.com https://www.comet.com/rodrigomiranda21/rmiranda-movie-reviews/74665d22480a4441a314ddb197f6599a



In [7]:
 params = {
     'layer-1-size': 16,
     'epochs': 10,
     'batch-size':512,
     'dropout': 0.16,
     }

# This will be logged in the Hyperparameters Tab in the Comet UI
exp.log_parameters(params)

**Load Model**

The model will be used for fine tuning. It will be the first "layer" of our NN.

In [8]:
model = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"

hub_layer = hub.KerasLayer(model, 
                           output_shape=[20], 
                           input_shape=[], 
                           dtype=tf.string, 
                           trainable=True)

In [ ]:
# hub_layer(train_examples[:3])

**Build Model**

Create Keras NN - Model + layer-1 + final layer (size 1) 

In [9]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(exp.get_parameter('layer-1-size'), activation='relu'))
model.add(tf.keras.layers.Dropout(exp.get_parameter('dropout')))
model.add(tf.keras.layers.Dense(1))

model.compile(optimizer='adam',
              loss=tf.losses.BinaryCrossentropy(from_logits=True),
              metrics=[tf.metrics.BinaryAccuracy(threshold=0.0, name='accuracy')])

In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 20)                400020    
                                                                 
 dense (Dense)               (None, 16)                336       
                                                                 
 dropout (Dropout)           (None, 16)                0         
                                                                 
 dense_1 (Dense)             (None, 1)                 17        
                                                                 
Total params: 400,373
Trainable params: 400,373
Non-trainable params: 0
_________________________________________________________________


**Train Model**

In [11]:
# After training the model you will see new Charts, Panels, Hyperparameters, Metrics and more in the Comet UI

model.fit(partial_x_train,
          partial_y_train,
          epochs=exp.get_parameter('epochs'),
          batch_size=exp.get_parameter('batch-size'),
          validation_data=(x_val, y_val),
          verbose=1)

COMET INFO: Ignoring automatic log_parameter('verbose') because 'keras:verbose' is in COMET_LOGGING_PARAMETERS_IGNORE


Epoch 1/10
30/30 [==============================] - 2s 39ms/step - loss: 1.0089 - accuracy: 0.5212 - val_loss: 0.6527 - val_accuracy: 0.6180
Epoch 2/10
30/30 [==============================] - 1s 33ms/step - loss: 0.7446 - accuracy: 0.6023 - val_loss: 0.6056 - val_accuracy: 0.6704
Epoch 3/10
30/30 [==============================] - 1s 34ms/step - loss: 0.6399 - accuracy: 0.6547 - val_loss: 0.5758 - val_accuracy: 0.6993
Epoch 4/10
30/30 [==============================] - 1s 34ms/step - loss: 0.5885 - accuracy: 0.6947 - val_loss: 0.5521 - val_accuracy: 0.7204
Epoch 5/10
30/30 [==============================] - 1s 34ms/step - loss: 0.5595 - accuracy: 0.7135 - val_loss: 0.5267 - val_accuracy: 0.7415
Epoch 6/10
30/30 [==============================] - 1s 34ms/step - loss: 0.5269 - accuracy: 0.7403 - val_loss: 0.5008 - val_accuracy: 0.7610
Epoch 7/10
30/30 [==============================] - 1s 34ms/step - loss: 0.4940 - accuracy: 0.7679 - val_loss: 0.4755 - val_accuracy: 0.7788
Epoch 8/10
30

Logging some things about this experiment:

In [12]:
# log any custom Metric
exp.log_metric('my_custom_metric', 3.14)
    
# log a dataset hash (Hyperparameter)
exp.log_dataset_hash(partial_x_train)

In [13]:
# Logs text: 10 examples from the test dataset with it's labels. These strings appear on the Text Tab in the Comet UI.

for i in range(10):
    exp.log_text(train_examples[i], metadata={"label": train_labels[i].item()})

In [14]:
# Logs the Confusion Matrix

preds = model.predict(test_examples)

# Function to have the labels in vector format    
def onehot(val):
    retval = [0, 0]
    tmp = (val[0] + 1) / 2
    tmp = int(round(tmp))
    tmp = max(min(1, tmp), 0)
    retval[tmp] = 1
    return retval
    
new_preds = [onehot(v) for v in preds]

# Function to show text examples on the Confusion Matrix GUI    
def index_to_example(index):
    text = test_examples[index]
    # data = experiment.log_text(text)
    return {"sample": text.decode(), 
            "assetId": None,
            "type": "string"}
    
exp.log_confusion_matrix(new_preds, 
                         test_labels, 
                         index_to_example_function=index_to_example,
                         file_name="movie-reviews")

782/782 [==============================] - 2s 2ms/step


COMET INFO: numpy not installed; using a slower winner_function for confusion matrix


{'web': 'https://www.comet.com/api/asset/download?assetId=f44f0f10a9a74eec8730b30c38efa711&experimentKey=74665d22480a4441a314ddb197f6599a',
 'api': 'https://www.comet.com/api/rest/v2/experiment/asset/get-asset?assetId=f44f0f10a9a74eec8730b30c38efa711&experimentKey=74665d22480a4441a314ddb197f6599a',
 'assetId': 'f44f0f10a9a74eec8730b30c38efa711'}

In [15]:
# Logs Model: The NN in h5 format

registry_model_name ='movie-reviews-nn'

model.save('models/movie-reviews-nn.h5')
exp.log_model(registry_model_name, 'models/movie-reviews-nn.h5')

{'web': 'https://www.comet.com/api/asset/download?assetId=c1062e00309f43149f1f8c9a53982ac6&experimentKey=74665d22480a4441a314ddb197f6599a',
 'api': 'https://www.comet.com/api/rest/v2/experiment/asset/get-asset?assetId=c1062e00309f43149f1f8c9a53982ac6&experimentKey=74665d22480a4441a314ddb197f6599a',
 'assetId': 'c1062e00309f43149f1f8c9a53982ac6'}

In [16]:
exp.end()

COMET INFO: ---------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.com/rodrigomiranda21/rmiranda-movie-reviews/74665d22480a4441a314ddb197f6599a
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     accuracy [10]                : (0.5212000012397766, 0.8229333162307739)
COMET INFO:     batch_accuracy [30]          : (0.478515625, 0.822265625)
COMET INFO:     batch_loss [30]              : (0.4046321213245392, 1.2748081684112549)
COMET INFO:     epoch_duration [10]          : (0.9927503999999772, 1.601389700000027)
COMET INFO:     loss [10]                    : (0.40698903799057007, 1.008898377418518)
COMET INFO:     my_custom_metric             : 3.14
COMET INFO:     val_accuracy [10]            : (0.6179999709129333, 0.8181999921798706)
COMET INFO:     val_loss [10]                : (0.408647745847702, 0.6526704

Register the model on Model Registry

In [15]:
api = API(api_key="EpnUTFvwqbgvmSpjWOTMBPlP5")

WORKSPACE = 'rodrigomiranda21'
PROJECT_NAME = 'rmiranda-movie-reviews'
EXP_KEY = exp.get_key()

api_exp = api.get_experiment(WORKSPACE, PROJECT_NAME, EXP_KEY)

api_exp.register_model(registry_model_name, version = "1.4.1")

COMET INFO: Successfully registered 'movie-reviews-nn', version '1.4.1' in workspace 'rodrigomiranda21'


{'registryModelId': 'OppQGGPpkHAvdP2d2237UafQ0',
 'registryModelItemId': 'O0oS5Q75Znl4kqYkdou96ZaS3'}